## Exemplo: Extração de informação de ficheiros de resultados de metabolómica.

[MassTRIX](http://www.masstrix.org), (_Mass TRanslator into Pathways_) [1] é um serviço online de tratamento de dados de metabolómica.

A funcionalidade primária é a identificação de compostos a partir de listas de massas e intensidades obtidas por análise de amostras biológicas por Espectrometria de Massa.

O resultado da identificação é disponibilizado em vários ficheiros de texto. Num dos formatos, cada linha do ficheiro diz respeito a um pico de massa e apresenta, de entre outros, os compostos identificados com aquela massa, bem como as anotações das vias celulares em que esses compostos podem estar envolvidos.

Pretende-se ilustrar o uso programático da leitura de ficheiros e as operações com _strings_ com um exemplo da **extração da informação contida num desses ficheiros**.

[1] K. Suhre and P. Schmitt-Kopplin (2008) MassTRIX: Mass TRanslator Into Pathways, _Nucleic Acids Research_, **36**, Web Server issue, W481-W484.

### Nota

Este notebook usa como exemplo um ficheiro chamado `masses.annotated.reformat.tsv`, mas este nome deve ser alterado para nome do ficheiro de resultados que é obtido de uma busca no MassTRIX.

### Extração da informação

Como resultado da análise são criados 2 dicionários:

In [1]:
compostos = {}
# Este dicionário associa a cada KEGG_cid um dicionário com

# 'nome'     : a string com o primeiro nome atribuído ao composto
# 'ids_vias' : uma lista de Ids das vias associadas ao composto
# 'vias'     : uma lista das vias associadas ao composto

vias = {}

# Este dicionário associa a cada KEGG Pathways Id (o Id da via) um dicionário com

# 'nome'          : a string com o nome da via
# 'ids_compostos' : uma lista de Ids dos compostos associados à via
# 'compostos'     : uma lista dos nomes dos compostos associados à via

In [2]:
nome_fich = 'masses.annotated.reformat.tsv' # alterar para o nome correto

with open(nome_fich) as f:
    linhas = f.read().splitlines()

headers = linhas[-1].split('\t')

for linha in linhas[:-1]:
    info = {}
    partes_linha = linha.split('\t')
    
    for h, c in zip(headers, partes_linha):
        info[h] = c.split('#')

    for comp, nomes, v_ids, v in zip(info['KEGG_cid'],
                                     info['KEGG_name'],
                                     info['KEGG Pathways'],
                                     info['KEGG Pathways descriptions']):

        if comp not in compostos: # se já existe, não voltar a adicionar ao dicionário
            
            # usar só o primeiro nome (e esperar que seja o melhor)
            nome = nomes.split(';')[0]
            
            compostos[comp] = {'nome': nome, 'ids_vias' : [], 'vias' : []}

            if v != 'null':
                compostos[comp]['ids_vias'] = v_ids.strip(';').split(';')
                compostos[comp]['vias'] = v.strip(';').split(';')


print(len(compostos), 'compostos')

for c in compostos: # c é o Id de cada composto
    for nome_via, via_id in zip(compostos[c]['vias'], compostos[c]['ids_vias']):
        nome_composto = compostos[c]['nome']
        if via_id not in vias:
            # é a primeira vez que é encontrada
            vias[via_id] = {'nome': nome_via, 
                            'ids_compostos': [c], 
                            'compostos': [nome_composto]}
        else:
            # já existe, acrescentar este composto
            vias[via_id]['ids_compostos'].append(c)
            vias[via_id]['compostos'].append(nome_composto)

print(len(vias), 'vias')

3605 compostos
150 vias


#### RETIRAR (ou adaptar) A CÉLULA SEGUINTE!
#### (verifica a existência de compostos e de vias, mas aplica-se aos dados do ficheiro de exemplo)

In [3]:
print('\n---- composto C00988: -----')
c = compostos['C00988']
print(c['nome'], '\n')
for id_v, v in zip(c['ids_vias'], c['vias']):
    print(id_v, ':', v)

print('\n---- composto C00079: -----')
c = compostos['C00079']
print(c['nome'], '\n')
for id_v, v in zip(c['ids_vias'], c['vias']):
    print(id_v, ':', v)
    
print('\n---- via ko00630: -----')
print(vias['ko00630']['nome'])
print('\nCompostos:')
for c in vias['ko00630']['compostos']:
    print(c)


---- composto C00988: -----
2-Phosphoglycolate 

ko00630 : Glyoxylate and dicarboxylate metabolism
ko01100 : Metabolic pathways

---- composto C00079: -----
L-Phenylalanine 

ko00360 : Phenylalanine metabolism
ko00400 : Phenylalanine, tyrosine and tryptophan biosynthesis
ko00940 : Phenylpropanoid biosynthesis
ko00960 : Tropane, piperidine and pyridine alkaloid biosynthesis
ko00966 : Glucosinolate biosynthesis
ko00970 : Aminoacyl-tRNA biosynthesis
ko01061 : Biosynthesis of phenylpropanoids
ko01063 : Biosynthesis of alkaloids derived from shikimate pathway
ko01064 : Biosynthesis of alkaloids derived from ornithine, lysine and nicotinic acid
ko01070 : Biosynthesis of plant hormones
ko01100 : Metabolic pathways
ko02010 : ABC transporters
map01060 : Biosynthesis of plant secondary metabolites

---- via ko00630: -----
Glyoxylate and dicarboxylate metabolism

Compostos:
2-Phosphoglycolate
3-Ethylmalate ([M-H]-)
(S)-Malate
2-Oxoglutarate
meso-Tartaric acid
(R,R)-Tartaric acid
2-Hydroxy-3-oxoa

### Criação de ficheiros com a informação extraída

Vamos criar ficheiros de texto com a informação extraída para mais tarde poder ser usada.

Vamos criar um ficheiro chamado `compostos.txt` e outro chamado `vias.txt`

As vias estão listadas por ordem decrescente do número de compostos encontrados.

In [4]:
with open('compostos.txt', 'w') as f:
    print('Id', 'nome', 'Ids vias', 'vias', file=f, sep='\t')
    
    for c, info in compostos.items():
        Ids_vias = '|'.join(info['ids_vias'])
        nomes_vias = '|'.join(info['vias'])
        print(c, info['nome'], Ids_vias, nomes_vias, file=f, sep='\t')

# ordenar as vias por número de compostos
contagens_compostos_em_vias = [(len(vias[k]['compostos']), k) for k in vias]
contagens_compostos_em_vias.sort(reverse=True)
# reverse=True indica que a ordenação é por ordem decrescente

with open('vias.txt', 'w') as f:
    print('Id via', 'nome via', 'n compostos', 'Ids compostos', 'compostos', file=f, sep='\t')
    
    for c, via in contagens_compostos_em_vias:
        infovia = vias[via]
        print(via, infovia['nome'], c, 
              '|'.join(infovia['ids_compostos']),
              '|'.join(infovia['compostos']), sep='\t', file=f)

**Verificar se correu bem, correndo o MS-Excel e abrindo estes ficheiros**